LOAD DATA 

In [14]:
import numpy as np
import matplotlib.pyplot as plt
data_path ='<PATH TO NPZ>/sample_1a2bc3012c9defca.npz'


In [15]:
info = np.load(data_path,allow_pickle=True)
all_agent_trajs        = info['all_agent']    
all_agent_trajs.shape  #[24 agents, 91 timesteps the interval between each timestep is 0.1 second,\
    #10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
#the valid flag is either 0 or 1. 1 means the valid, 0 means invalid
obj_types              = info['object_type']
lane_polylines         = info['lane']           #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
road_polylines         = info['road_polylines'] #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
crosswalk_polylines    = info['crosswalk']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
speed_bump_polylines   = info['speed_bump']     #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
stop_signs_polylines   = info['stop_sign']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
drive_way_polylines    = info['drive_way']      #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]


We select the 11th timestep as the current timestep 

In [16]:
all_agent_current = all_agent_trajs[:,10]
all_agent_current.shape

(24, 10)

In [17]:
#ground truth for the future 1s,2s,3s can be easily derived by :
predict_horizon = 10 #20,30
all_gt_future=all_agent_trajs[:,11:11+predict_horizon]
all_gt_future.shape

(24, 10, 10)

Index of target agnet to preict

In [18]:
tracks        = info['predict_list']  
tracks
agents_to_predict = all_agent_trajs[tracks]
agents_to_predict.shape

(8, 91, 10)

The homework should be done on these 8 agents.

Q1.Using constant velocity model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE.  

Q2.Using constant acceleration model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

Using Constant Turn Rate and Velocity(CTRV) model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

Briefly discuss the observations from your results. 

If you want to visualize the scenario here are some simple demo codes for you.

    'TYPE_FREEWAY': 1,
    'TYPE_SURFACE_STREET': 2,
    'TYPE_BIKE_LANE': 3,

    # for roadline
    'TYPE_UNKNOWN': -1,
    'TYPE_BROKEN_SINGLE_WHITE': 6,
    'TYPE_SOLID_SINGLE_WHITE': 7,
    'TYPE_SOLID_DOUBLE_WHITE': 8,
    'TYPE_BROKEN_SINGLE_YELLOW': 9,
    'TYPE_BROKEN_DOUBLE_YELLOW': 10,
    'TYPE_SOLID_SINGLE_YELLOW': 11,
    'TYPE_SOLID_DOUBLE_YELLOW': 12,
    'TYPE_PASSING_DOUBLE_YELLOW': 13,

    # for roadedge
    'TYPE_ROAD_EDGE_BOUNDARY': 15,
    'TYPE_ROAD_EDGE_MEDIAN': 16,

    # for stopsign
    'TYPE_STOP_SIGN': 17,

    # for crosswalk
    'TYPE_CROSSWALK': 18,

    # for speed bump
    'TYPE_SPEED_BUMP': 19,
    
    # for driveway
    'TYPE_DRIVEWAY': 20,

In [21]:
sdc_current_state = all_agent_trajs[tracks[0]][11]
ax = plt.gca()
fig = plt.gcf()
fig.set_facecolor('xkcd:grey') 
ax.set_facecolor('xkcd:grey')
for polyline in road_polylines:
    map_type = polyline[0,6]
    if map_type == 6:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
    elif map_type == 7:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 8:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 9:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 10:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 11:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 12:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 13:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
    elif map_type == 15:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
    elif map_type == 16:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
ax.axis([-70+ sdc_current_state[0], 70+ sdc_current_state[0], -70+ sdc_current_state[1], 70 + sdc_current_state[1]])

filename = './viz.png' 
plt.savefig(filename)       
plt.close()